In [1]:
a = ["a","b","c","d"]
with open("a.txt", "w") as f:
    f.write('\n'.join(a))

洪暉鈞
臺北醫學大學  大數據科技及管理研究所
hch@tmu.edu.tw

# 【網路爬蟲 web crawler】應用篇

### BeautifulSoup 相關應用 
[說明](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)

##### BeautifulSoup 篩選網頁圖片連結

In [1]:
# BeautifulSoup 篩選網頁圖片連結part1
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse

url = "http://www.tmu.edu.tw/main.php"
domain = "{}://{}".format(urlparse(url).scheme, urlparse(url).hostname)

html = requests.get(url)
html.encoding = 'utf-8'
sp = BeautifulSoup(html.text, 'html.parser')
all_links = sp.find_all(["a","img"])

In [2]:
all_links

[<a accesskey="Q" href="/accesskey.php" id="accesskey_top" title="捷徑工具列">:::</a>,
 <a href="/main.php">回首頁</a>,
 <a href="https://tmu.pure.elsevier.com" target="_blank">學術庫</a>,
 <a href="/sitemap/site_map.php">網站地圖</a>,
 <a href="http://www.tmu.edu.tw/english/" target="_blank">English </a>,
 <a href="/students/super_pages.php?ID=students">學生</a>,
 <a href="/staff/super_pages.php?ID=staff">教職員</a>,
 <a href="/parents/super_pages.php?ID=parents">家長</a>,
 <a href="/alumni/super_pages.php?ID=alumni">校友</a>,
 <a href="/main.php" title="回首頁">
 <img alt="LOGO" src="/css/logo.jpg" title="回首頁"/>
 </a>,
 <img alt="LOGO" src="/css/logo.jpg" title="回首頁"/>,
 <a accesskey="C" href="/accesskey.php" id="accesskey_content" title="內容區塊">:::</a>,
 <a href="/app/news.php?Sn=2490"> 108學年度特殊選才招生簡章公告 </a>,
 <a href="/app/news.php?Sn=2485"> 108學年度碩、博士班甄試入學、EMBA碩專班考試入學簡章及招生日程公告 </a>,
 <a href="/app/news.php?Sn=2483">  108學年度碩、博士班甄試入學系所學位學程說明會即日起開放網路報名! </a>,
 <a href="/app/news.php?Sn=2465"> 107學年度轉系榜單公告 </a>

In [20]:
# BeautifulSoup 篩選網頁圖片連結part2

for link in all_links:
    src = link.get('src')
    href = link.get('href')
    targets = [src, href]
    for t in targets:
        if t != None and ('.jpg' in t or '.png' in t):
            if t.startswith('http'): print(t)
            else: print(domain+t)

http://www.tmu.edu.tw/css/logo.jpg
http://www.tmu.edu.tw/files/news/2381_3d91e4d0.jpg
http://www.tmu.edu.tw/files/news/2448_97ca658d.jpg
http://www.tmu.edu.tw/files/news/2460_3f29f98c.jpg
http://www.tmu.edu.tw/files/news/2479_2fd4aa36.png
http://www.tmu.edu.tw/files/news/2436_7ab5d820.jpg
http://www.tmu.edu.tw/files/news/2358_55a2cd1d.jpg
http://www.tmu.edu.tw/files/news/2386_0c27dc85.jpg
http://www.tmu.edu.tw/files/news/2489_6923efe8.png


##### BeautifulSoup 篩選網頁圖片連結==>下載

In [21]:
# BeautifulSoup 篩選網頁圖片連結part1
from bs4 import BeautifulSoup
import requests, os
from urllib.parse import urlparse
from urllib.request import urlopen

url = "http://www.tmu.edu.tw/main.php"
domain = "{}://{}".format(urlparse(url).scheme, urlparse(url).hostname)
html = requests.get(url)
html.encoding = 'utf-8'
text = html.text
sp = BeautifulSoup(text, 'html.parser')
all_links = sp.find_all(['a','img'])


In [23]:
# BeautifulSoup 篩選網頁圖片連結 下載功能

for link in all_links:
    src = link.get('src')
    href = link.get('href')
    targets = [src, href]
    
    image_dir = url.split('/')[-1]
    if not os.path.exists(image_dir): os.mkdir(image_dir)
    
    for t in targets:
        if t != None and ('.jpg' in t or '.png' in t):
            if t.startswith('http'): full_path = t
            else: full_path = domain+t
            
            print(full_path)
            
            filename = full_path.split('/')[-1]
            ext = filename.split('.')[-1]
            filename = filename.split('.')[-2]
            
            if 'jpg' in ext: filename = filename + '.jpg'
            else: filename = filename + '.png'
            
            image = urlopen(full_path)
            with open(os.path.join(image_dir,filename),'wb') as fp:
                fp.write(image.read())

http://www.tmu.edu.tw/css/logo.jpg
http://www.tmu.edu.tw/files/news/2381_3d91e4d0.jpg
http://www.tmu.edu.tw/files/news/2479_2fd4aa36.png
http://www.tmu.edu.tw/files/news/2460_3f29f98c.jpg
http://www.tmu.edu.tw/files/news/2436_7ab5d820.jpg
http://www.tmu.edu.tw/files/news/2386_0c27dc85.jpg
http://www.tmu.edu.tw/files/news/2358_55a2cd1d.jpg
http://www.tmu.edu.tw/files/news/2489_6923efe8.png
http://www.tmu.edu.tw/files/news/2448_97ca658d.jpg


##### 中油網站油價爬蟲

In [39]:
from bs4 import BeautifulSoup
import requests

#url = 'http://new.cpc.com.tw/division/mb/oil-more4.aspx'
url = "https://z4rokvnz43i149vk93nzfw-on.drv.tw/www/oil.html"
html = requests.get(url).text
sp = BeautifulSoup(html, 'html.parser')

data = sp.find_all('span', id ='Showtd')
rows = data[0].find_all('tr')

prices = list()
for row in rows:
    cols = row.find_all('td')
    if len(cols[1].text) > 0:
        item = [cols[i].text for i in range(4)] # 這樣寫比原本簡潔
        prices.append(item)
for p in prices:
    print(p)



['2018/11/12', '28.5', '30.0', '32.0']
['2018/11/05', '29.2', '30.7', '32.7']
['2018/10/29', '29.5', '31.0', '33.0']
['2018/10/22', '29.9', '31.4', '33.4']
['2018/10/01', '30.0', '31.5', '33.5']
['2018/09/17', '29.5', '31.0', '33.0']
['2018/09/10', '29.3', '30.8', '32.8']
['2018/09/03', '29.2', '30.7', '32.7']
['2018/08/27', '28.8', '30.3', '32.3']
['2018/08/20', '28.5', '30.0', '32.0']
['2018/08/13', '28.6', '30.1', '32.1']
['2018/07/30', '28.8', '30.3', '32.3']
['2018/07/23', '28.5', '30.0', '32.0']
['2018/07/16', '28.9', '30.4', '32.4']
['2018/07/09', '29.1', '30.6', '32.6']
['2018/07/02', '28.9', '30.4', '32.4']
['2018/06/25', '28.5', '30.0', '32.0']
['2018/06/18', '28.7', '30.2', '32.2']
['2018/06/11', '28.6', '30.1', '32.1']
['2018/06/04', '28.8', '30.3', '32.3']
['2018/05/28', '29.2', '30.7', '32.7']
['2018/05/21', '29.0', '30.5', '32.5']
['2018/05/14', '28.7', '30.2', '32.2']
['2018/05/07', '28.1', '29.6', '31.6']
['2018/04/30', '28.2', '29.7', '31.7']
['2018/04/23', '27.8', '2

In [40]:
data[0].find_all('tr')[1].find_all('td')[3].text

'32.0'

##### 中油網站油價爬蟲+網頁應用

In [41]:
from bs4 import BeautifulSoup
import requests
pre_html = '''
<!DOCTYPE html>
<html>
<head>
<meta charset='utf-8'>
<title>中油油價歷史資料</title>
</head>
<body>
<h2>中油油價歷史資料（取自中油官方網站）</h2>
<table width=600 border=1>
<tr><td>日期</td><td>92無鉛</td><td>95無鉛</td><td>98無鉛</td></tr>
'''

post_html = '''
</table>
</body>
</html>
'''



In [43]:
url = 'https://z4rokvnz43i149vk93nzfw-on.drv.tw/www/oil.html'
html = requests.get(url).text
sp = BeautifulSoup(html, 'html.parser')
data = sp.find_all('span', {'id':'Showtd'})
rows = data[0].find_all('tr')
prices = list()

for row in rows:
    cols = row.find_all('td')
    if len(cols[1].text) > 0:
        item = [cols[i].text for i in range(4)] # 這樣寫比原本簡潔
        prices.append(item)

html_body = ''

for p in prices:
    html_body += "<tr><td>{}</td><td>{}</td><td>{}</td><td>{}</td></tr>".format(p[0],p[1],p[2],p[3])

html_file = pre_html + html_body + post_html

with open('oilprice.html','w', encoding='utf-8') as fp:
    fp.write(html_file)


### 進階應用：博客來網路書店搜尋結果

In [44]:
# 載入模組

import requests
from bs4 import BeautifulSoup

res = requests.get("http://search.books.com.tw/search/query/key/%E6%A9%9F%E5%99%A8%E5%AD%B8%E7%BF%92/cat/all")
soup = BeautifulSoup(res.text,'html.parser')

In [45]:
#標題說明
soup.title.string

'博客來-目前您搜尋的關鍵字為: 機器學習'

In [48]:
#書名說明
# <img class='itemcov' alt='書名'....
soup.select("img[class='itemcov']")[1]['alt']

'Python機器學習(第二版)'

In [52]:
#作者說明
soup.select("li[class='item']")[0].select("a[rel='go_author']")[0].text

'立石賢吾'

In [53]:
#價格說明
soup.select("span[class='price']")[0].select('b')

[<b>79</b>, <b>356</b>]

In [59]:
import pandas as pd
pd.Series(soup.select("img[class='itemcov']")[0]['alt'])

0    練好機器學習的基本功：用Python進行基礎數學理論的實作
dtype: object

In [85]:
#爬書名，抓下來放到 names 
#soup.select("img[class='itemcov']")[0]['alt']

import pandas as pd
names = pd.Series()
for i in soup.select("img[class='itemcov']"):
    names = names.append(pd.Series([i['alt']])).reset_index(drop=True)  #加到pd.Series

In [83]:
names

0            練好機器學習的基本功：用Python進行基礎數學理論的實作
1                          Python機器學習(第二版)
2                       機器學習|工作現場的評估、導入與實作
3                    機器學習入門：從玻爾茲曼機器學習到深度學習
4                                初探機器學習演算法
5                       實戰機器學習：以深度學習演算企業資料
6      Python+Spark 2.0+Hadoop機器學習與大數據分析實戰
7                            機器學習：探索人工智慧關鍵
8     R語言資料分析：從機器學習、資料探勘、文字探勘到巨量資料分析 [第二版]
9          今天不學機器學習，明天就被機器取代：從Python入手+演算法
10                              白話大數據與機器學習
11                              Python機器學習
12                  Spark 2.x 大數據分析與機器學習實戰
13          大數據的下一步：Spark MLlib機器學習實戰技巧大公開
14                  mBot入門與實習：STEM整合式機器人學習
15                          實戰機器學習：使用Spark
16                改變未來20年最重要的20個視覺機器學習理論深讀
17                        AI及機器學習的經脈：演算法新解
18         Google就是這樣猜中你的心：用機器學習及演算法分析文字語意
19      比Hadoop+Python還強：Spark MLlib機器學習實作
dtype: object

In [118]:
#作者結構說明
#soup.select("li[class='item']")[0].select("a[rel='go_author']")[0].text

authors = pd.Series()
aus = []
#soup.select("li[class='item']")[0].select("a[rel='go_author']")[0].text
for i in soup.select("li[class='item']"):
    au = []
    for a in i.select("a[rel='go_author']"):
        au.append(a.text)
    aus.append(au)
    
aus   

[['立石賢吾'],
 ['Sebastian Raschka', 'Vahid Mirjalili', '劉立民', '吳建華'],
 ['中山心太', '有賀康顕', '西林孝', '許郁文'],
 ['大関真之', '許郁文'],
 ['Giuseppe Bonaccorso', '賴屹民'],
 ['李博', '賴榮樞'],
 ['林大貴'],
 ['Ethem Alpaydin', '劉一慧', '蕭俊傑'],
 ['李仁鐘', '李秋緣'],
 ['鄭捷'],
 ['尹會生', '衛崢', '高揚', '萬娟/插畫設計'],
 ['Sebastian Raschka', '劉立民', '吳建華'],
 ['Romeo Kienzler', '賴裕文'],
 ['王曉華'],
 ['吳志宏'],
 ['Manpreet Singh Ghotra', 'Nick Pentreath', 'Rajdeep Dua', '江良志'],
 ['劉通', '張政', '張立甯', '方宇強', '李沛秦', '沈傑', '王勇', '胡俊', '興軍亮', '謝劍斌', '譚筠', '閆瑋'],
 ['劉新宇'],
 ['祁瑞華'],
 ['王曉華']]

In [128]:
# 爬作者，抓下來放到authors
# soup.select("li[class='item']")[0].select("a[rel='go_author']")[0].text

authors = pd.Series()

for i in soup.select("li[class='item']"):
    au = []
    for a in i.select("a[rel='go_author']"):
        au.append(a.text)
    authors = authors.append(pd.Series(", ".join(au))).reset_index(drop=True)
    

## 我的寫法（下面），先存在 list 最後在轉成 pd.Series 也可以

In [130]:
aus = []

for i in soup.select("li[class='item']"):
    au = []
    for a in i.select("a[rel='go_author']"):
        au.append(a.text)
    aus.append(", ".join(au))
    
authors = pd.Series(aus)

In [131]:
authors

0                                                  立石賢吾
1          Sebastian Raschka, Vahid Mirjalili, 劉立民, 吳建華
2                                  中山心太, 有賀康顕, 西林孝, 許郁文
3                                             大関真之, 許郁文
4                              Giuseppe Bonaccorso, 賴屹民
5                                               李博, 賴榮樞
6                                                   林大貴
7                              Ethem Alpaydin, 劉一慧, 蕭俊傑
8                                              李仁鐘, 李秋緣
9                                                    鄭捷
10                                 尹會生, 衛崢, 高揚, 萬娟/插畫設計
11                          Sebastian Raschka, 劉立民, 吳建華
12                                  Romeo Kienzler, 賴裕文
13                                                  王曉華
14                                                  吳志宏
15    Manpreet Singh Ghotra, Nick Pentreath, Rajdeep...
16    劉通, 張政, 張立甯, 方宇強, 李沛秦, 沈傑, 王勇, 胡俊, 興軍亮, 謝劍斌, 譚...
17                                              

In [132]:
# 價格結構說明
# soup.select("span[class='price']")[0].select('b')

for price in soup.select("span[class='price']"):
    print(price.select('b')) #list type

[<b>79</b>, <b>356</b>]
[<b>79</b>, <b>545</b>]
[<b>79</b>, <b>458</b>]
[<b>95</b>, <b>361</b>]
[<b>95</b>, <b>456</b>]
[<b>95</b>, <b>428</b>]
[<b>9</b>, <b>612</b>]
[<b>79</b>, <b>316</b>]
[<b>9</b>, <b>468</b>]
[<b>9</b>, <b>531</b>]
[<b>95</b>, <b>456</b>]
[<b>9</b>, <b>522</b>]
[<b>95</b>, <b>618</b>]
[<b>9</b>, <b>468</b>]
[<b>79</b>, <b>332</b>]
[<b>95</b>, <b>589</b>]
[<b>9</b>, <b>441</b>]
[<b>9</b>, <b>621</b>]
[<b>9</b>, <b>441</b>]
[<b>9</b>, <b>432</b>]
[<b>79</b>, <b>356</b>]
[<b>79</b>, <b>458</b>]
[<b>79</b>, <b>490</b>]
[<b>79</b>, <b>466</b>]
[<b>79</b>, <b>616</b>]


In [140]:
# 爬取價格，抓下來放到prices

i = 0
prices = pd.Series()
for price in soup.select("span[class='price']"):
    if(i<names.size):
        if(len(price.select('b'))==1): 
            prices = prices.append(pd.Series([price.select('b')[0].string])).reset_index(drop=True) # .string取tag<b>中的文字內容
        elif(len(price.select('b'))==2): 
            prices = prices.append(pd.Series([price.select('b')[1].string])).reset_index(drop=True) # .string取tag<b>中的文字內容
        else:
            break
    i += 1

## 我的寫法（下面）

In [165]:
len(soup.select("span[class='price']")[0].select('b')) in [1,2]

True

In [168]:
#soup.select("span[class='price']")[0].select('b')[-1].text

prices = []
i = 0
size = names.size
for price in soup.select("span[class='price']"):
    if i < size and len(price.select('b')) in [1,2]:
        prices.append(price.select('b')[-1].text) #list type
    i += 1
prices = pd.Series(prices)

In [169]:
prices

0     356
1     545
2     458
3     361
4     456
5     428
6     612
7     316
8     468
9     531
10    456
11    522
12    618
13    468
14    332
15    589
16    441
17    621
18    441
19    432
dtype: object

In [170]:
# 合併成DataFrame
df = pd.DataFrame({'書名':names,'作者':authors,'價格':prices})
df[['書名','作者','價格']]

,書名,作者,價格
0,練好機器學習的基本功：用Python進行基礎數學理論的實作,立石賢吾,356
1,Python機器學習(第二版),"Sebastian Raschka, Vahid Mirjalili, 劉立民, 吳建華",545
2,機器學習|工作現場的評估、導入與實作,"中山心太, 有賀康顕, 西林孝, 許郁文",458
3,機器學習入門：從玻爾茲曼機器學習到深度學習,"大関真之, 許郁文",361
4,初探機器學習演算法,"Giuseppe Bonaccorso, 賴屹民",456
5,實戰機器學習：以深度學習演算企業資料,"李博, 賴榮樞",428
6,Python+Spark 2.0+Hadoop機器學習與大數據分析實戰,林大貴,612
7,機器學習：探索人工智慧關鍵,"Ethem Alpaydin, 劉一慧, 蕭俊傑",316
8,R語言資料分析：從機器學習、資料探勘、文字探勘到巨量資料分析 [第二版],"李仁鐘, 李秋緣",468
9,今天不學機器學習，明天就被機器取代：從Python入手+演算法,鄭捷,531


### 本周作業：
請以一頁網頁之內容為例進行爬蟲，選擇其特定內容，做成表格之特定欄位存成dataframe顯示。

In [1]:
#請說明要爬蟲的網址

# 中選會107年公民投票
### http://referendum.2018.nat.gov.tw/pc/zh_TW/00/00000000000000000.html

In [ ]:
#爬蟲程式

In [36]:
import requests
from bs4 import BeautifulSoup
import re

# 由於該網頁有些 html 是透過 javascript render 的，直接使用 requests 會缺漏部分內容
# 解決方法是利用瀏覽器先把 html 載下來，再讀檔

# 請先自行下載 http://referendum.2018.nat.gov.tw/pc/zh_TW/00/00000000000000000.html

with open("107年全民性公民投票.html",'r',encoding="utf-8") as html:
    soup = BeautifulSoup(html, "html.parser")

In [67]:
# 抓取所有鄉鎮市區獨立頁面的連結
links = []

items = soup.find_all("div", id=re.compile("item[0-9]+"))
items = items[1:] #第一個爲全國統計，故刪除之

for item in items:
    links.append(item.find("a")["href"])

In [ ]:
#抓下來的資料整理成dataframe

In [121]:
data = [] # 2-D array, rows:鄉鎮市區, columns:columns

for link in links:
    print(link)
    page = requests.get(link) # 取得該鄉鎮市區的獨立頁面
    sp = BeautifulSoup(page.text,"html.parser")
    
    values = []

    b = sp.find("b").text.split("\xa0")
    values.extend((b[0],b[3][:3],b[3][3:])) # 公投案,縣市,鄉鎮市區

    trs = sp.find_all("tr", class_="trT")
    for tr in trs:
        tds = tr.find_all("td")
        for td in tds:
            values.append(td.text)
    
    data.append(values)

http://referendum.2018.nat.gov.tw/pc/zh_TW/01/63000000100000000.html
http://referendum.2018.nat.gov.tw/pc/zh_TW/01/63000000200000000.html
http://referendum.2018.nat.gov.tw/pc/zh_TW/01/63000000300000000.html
http://referendum.2018.nat.gov.tw/pc/zh_TW/01/63000000400000000.html
http://referendum.2018.nat.gov.tw/pc/zh_TW/01/63000000500000000.html
http://referendum.2018.nat.gov.tw/pc/zh_TW/01/63000000600000000.html
http://referendum.2018.nat.gov.tw/pc/zh_TW/01/63000000700000000.html
http://referendum.2018.nat.gov.tw/pc/zh_TW/01/63000000800000000.html
http://referendum.2018.nat.gov.tw/pc/zh_TW/01/63000000900000000.html
http://referendum.2018.nat.gov.tw/pc/zh_TW/01/63000001000000000.html
http://referendum.2018.nat.gov.tw/pc/zh_TW/01/63000001100000000.html
http://referendum.2018.nat.gov.tw/pc/zh_TW/01/63000001200000000.html
http://referendum.2018.nat.gov.tw/pc/zh_TW/01/65000000100000000.html
http://referendum.2018.nat.gov.tw/pc/zh_TW/01/65000000200000000.html
http://referendum.2018.nat.gov.tw/

http://referendum.2018.nat.gov.tw/pc/zh_TW/01/67000003700000000.html
http://referendum.2018.nat.gov.tw/pc/zh_TW/01/64000000100000000.html
http://referendum.2018.nat.gov.tw/pc/zh_TW/01/64000000200000000.html
http://referendum.2018.nat.gov.tw/pc/zh_TW/01/64000000300000000.html
http://referendum.2018.nat.gov.tw/pc/zh_TW/01/64000000400000000.html
http://referendum.2018.nat.gov.tw/pc/zh_TW/01/64000000500000000.html
http://referendum.2018.nat.gov.tw/pc/zh_TW/01/64000000600000000.html
http://referendum.2018.nat.gov.tw/pc/zh_TW/01/64000000700000000.html
http://referendum.2018.nat.gov.tw/pc/zh_TW/01/64000000800000000.html
http://referendum.2018.nat.gov.tw/pc/zh_TW/01/64000000900000000.html
http://referendum.2018.nat.gov.tw/pc/zh_TW/01/64000001000000000.html
http://referendum.2018.nat.gov.tw/pc/zh_TW/01/64000001100000000.html
http://referendum.2018.nat.gov.tw/pc/zh_TW/01/64000001200000000.html
http://referendum.2018.nat.gov.tw/pc/zh_TW/01/64000001300000000.html
http://referendum.2018.nat.gov.tw/

http://referendum.2018.nat.gov.tw/pc/zh_TW/01/10009001100000000.html
http://referendum.2018.nat.gov.tw/pc/zh_TW/01/10009001200000000.html
http://referendum.2018.nat.gov.tw/pc/zh_TW/01/10009001300000000.html
http://referendum.2018.nat.gov.tw/pc/zh_TW/01/10009001400000000.html
http://referendum.2018.nat.gov.tw/pc/zh_TW/01/10009001500000000.html
http://referendum.2018.nat.gov.tw/pc/zh_TW/01/10009001600000000.html
http://referendum.2018.nat.gov.tw/pc/zh_TW/01/10009001700000000.html
http://referendum.2018.nat.gov.tw/pc/zh_TW/01/10009001800000000.html
http://referendum.2018.nat.gov.tw/pc/zh_TW/01/10009001900000000.html
http://referendum.2018.nat.gov.tw/pc/zh_TW/01/10009002000000000.html
http://referendum.2018.nat.gov.tw/pc/zh_TW/01/10010000100000000.html
http://referendum.2018.nat.gov.tw/pc/zh_TW/01/10010000200000000.html
http://referendum.2018.nat.gov.tw/pc/zh_TW/01/10010000300000000.html
http://referendum.2018.nat.gov.tw/pc/zh_TW/01/10010000400000000.html
http://referendum.2018.nat.gov.tw/

http://referendum.2018.nat.gov.tw/pc/zh_TW/01/10017000300000000.html
http://referendum.2018.nat.gov.tw/pc/zh_TW/01/10017000400000000.html
http://referendum.2018.nat.gov.tw/pc/zh_TW/01/10017000500000000.html
http://referendum.2018.nat.gov.tw/pc/zh_TW/01/10017000600000000.html
http://referendum.2018.nat.gov.tw/pc/zh_TW/01/10017000700000000.html
http://referendum.2018.nat.gov.tw/pc/zh_TW/01/10018000100000000.html
http://referendum.2018.nat.gov.tw/pc/zh_TW/01/10018000200000000.html
http://referendum.2018.nat.gov.tw/pc/zh_TW/01/10018000300000000.html
http://referendum.2018.nat.gov.tw/pc/zh_TW/01/10020000100000000.html
http://referendum.2018.nat.gov.tw/pc/zh_TW/01/10020000200000000.html
http://referendum.2018.nat.gov.tw/pc/zh_TW/02/63000000100000000.html
http://referendum.2018.nat.gov.tw/pc/zh_TW/02/63000000200000000.html
http://referendum.2018.nat.gov.tw/pc/zh_TW/02/63000000300000000.html
http://referendum.2018.nat.gov.tw/pc/zh_TW/02/63000000400000000.html
http://referendum.2018.nat.gov.tw/

http://referendum.2018.nat.gov.tw/pc/zh_TW/02/67000002900000000.html
http://referendum.2018.nat.gov.tw/pc/zh_TW/02/67000003000000000.html
http://referendum.2018.nat.gov.tw/pc/zh_TW/02/67000003100000000.html
http://referendum.2018.nat.gov.tw/pc/zh_TW/02/67000003200000000.html
http://referendum.2018.nat.gov.tw/pc/zh_TW/02/67000003300000000.html
http://referendum.2018.nat.gov.tw/pc/zh_TW/02/67000003400000000.html
http://referendum.2018.nat.gov.tw/pc/zh_TW/02/67000003500000000.html
http://referendum.2018.nat.gov.tw/pc/zh_TW/02/67000003600000000.html
http://referendum.2018.nat.gov.tw/pc/zh_TW/02/67000003700000000.html
http://referendum.2018.nat.gov.tw/pc/zh_TW/02/64000000100000000.html
http://referendum.2018.nat.gov.tw/pc/zh_TW/02/64000000200000000.html
http://referendum.2018.nat.gov.tw/pc/zh_TW/02/64000000300000000.html
http://referendum.2018.nat.gov.tw/pc/zh_TW/02/64000000400000000.html
http://referendum.2018.nat.gov.tw/pc/zh_TW/02/64000000500000000.html
http://referendum.2018.nat.gov.tw/

...

http://referendum.2018.nat.gov.tw/pc/zh_TW/09/10005000700000000.html
http://referendum.2018.nat.gov.tw/pc/zh_TW/09/10005000800000000.html
http://referendum.2018.nat.gov.tw/pc/zh_TW/09/10005000900000000.html
http://referendum.2018.nat.gov.tw/pc/zh_TW/09/10005001000000000.html
http://referendum.2018.nat.gov.tw/pc/zh_TW/09/10005001100000000.html
http://referendum.2018.nat.gov.tw/pc/zh_TW/09/10005001200000000.html
http://referendum.2018.nat.gov.tw/pc/zh_TW/09/10005001300000000.html
http://referendum.2018.nat.gov.tw/pc/zh_TW/09/10005001400000000.html
http://referendum.2018.nat.gov.tw/pc/zh_TW/09/10005001500000000.html
http://referendum.2018.nat.gov.tw/pc/zh_TW/09/10005001600000000.html
http://referendum.2018.nat.gov.tw/pc/zh_TW/09/10005001700000000.html
http://referendum.2018.nat.gov.tw/pc/zh_TW/09/10005001800000000.html
http://referendum.2018.nat.gov.tw/pc/zh_TW/09/10007000100000000.html
http://referendum.2018.nat.gov.tw/pc/zh_TW/09/10007000200000000.html
http://referendum.2018.nat.gov.tw/

http://referendum.2018.nat.gov.tw/pc/zh_TW/09/10002000100000000.html
http://referendum.2018.nat.gov.tw/pc/zh_TW/09/10002000200000000.html
http://referendum.2018.nat.gov.tw/pc/zh_TW/09/10002000300000000.html
http://referendum.2018.nat.gov.tw/pc/zh_TW/09/10002000400000000.html
http://referendum.2018.nat.gov.tw/pc/zh_TW/09/10002000500000000.html
http://referendum.2018.nat.gov.tw/pc/zh_TW/09/10002000600000000.html
http://referendum.2018.nat.gov.tw/pc/zh_TW/09/10002000700000000.html
http://referendum.2018.nat.gov.tw/pc/zh_TW/09/10002000800000000.html
http://referendum.2018.nat.gov.tw/pc/zh_TW/09/10002000900000000.html
http://referendum.2018.nat.gov.tw/pc/zh_TW/09/10002001000000000.html
http://referendum.2018.nat.gov.tw/pc/zh_TW/09/10002001100000000.html
http://referendum.2018.nat.gov.tw/pc/zh_TW/09/10002001200000000.html
http://referendum.2018.nat.gov.tw/pc/zh_TW/09/10015000100000000.html
http://referendum.2018.nat.gov.tw/pc/zh_TW/09/10015000200000000.html
http://referendum.2018.nat.gov.tw/

http://referendum.2018.nat.gov.tw/pc/zh_TW/10/68000001200000000.html
http://referendum.2018.nat.gov.tw/pc/zh_TW/10/68000001300000000.html
http://referendum.2018.nat.gov.tw/pc/zh_TW/10/66000000100000000.html
http://referendum.2018.nat.gov.tw/pc/zh_TW/10/66000000200000000.html
http://referendum.2018.nat.gov.tw/pc/zh_TW/10/66000000300000000.html
http://referendum.2018.nat.gov.tw/pc/zh_TW/10/66000000400000000.html
http://referendum.2018.nat.gov.tw/pc/zh_TW/10/66000000500000000.html
http://referendum.2018.nat.gov.tw/pc/zh_TW/10/66000000600000000.html
http://referendum.2018.nat.gov.tw/pc/zh_TW/10/66000000700000000.html
http://referendum.2018.nat.gov.tw/pc/zh_TW/10/66000000800000000.html
http://referendum.2018.nat.gov.tw/pc/zh_TW/10/66000000900000000.html
http://referendum.2018.nat.gov.tw/pc/zh_TW/10/66000001000000000.html
http://referendum.2018.nat.gov.tw/pc/zh_TW/10/66000001100000000.html
http://referendum.2018.nat.gov.tw/pc/zh_TW/10/66000001200000000.html
http://referendum.2018.nat.gov.tw/

http://referendum.2018.nat.gov.tw/pc/zh_TW/10/10005000100000000.html
http://referendum.2018.nat.gov.tw/pc/zh_TW/10/10005000200000000.html
http://referendum.2018.nat.gov.tw/pc/zh_TW/10/10005000300000000.html
http://referendum.2018.nat.gov.tw/pc/zh_TW/10/10005000400000000.html
http://referendum.2018.nat.gov.tw/pc/zh_TW/10/10005000500000000.html
http://referendum.2018.nat.gov.tw/pc/zh_TW/10/10005000600000000.html
http://referendum.2018.nat.gov.tw/pc/zh_TW/10/10005000700000000.html
http://referendum.2018.nat.gov.tw/pc/zh_TW/10/10005000800000000.html
http://referendum.2018.nat.gov.tw/pc/zh_TW/10/10005000900000000.html
http://referendum.2018.nat.gov.tw/pc/zh_TW/10/10005001000000000.html
http://referendum.2018.nat.gov.tw/pc/zh_TW/10/10005001100000000.html
http://referendum.2018.nat.gov.tw/pc/zh_TW/10/10005001200000000.html
http://referendum.2018.nat.gov.tw/pc/zh_TW/10/10005001300000000.html
http://referendum.2018.nat.gov.tw/pc/zh_TW/10/10005001400000000.html
http://referendum.2018.nat.gov.tw/

http://referendum.2018.nat.gov.tw/pc/zh_TW/10/10013002700000000.html
http://referendum.2018.nat.gov.tw/pc/zh_TW/10/10013002800000000.html
http://referendum.2018.nat.gov.tw/pc/zh_TW/10/10013002900000000.html
http://referendum.2018.nat.gov.tw/pc/zh_TW/10/10013003000000000.html
http://referendum.2018.nat.gov.tw/pc/zh_TW/10/10013003100000000.html
http://referendum.2018.nat.gov.tw/pc/zh_TW/10/10013003200000000.html
http://referendum.2018.nat.gov.tw/pc/zh_TW/10/10013003300000000.html
http://referendum.2018.nat.gov.tw/pc/zh_TW/10/10002000100000000.html
http://referendum.2018.nat.gov.tw/pc/zh_TW/10/10002000200000000.html
http://referendum.2018.nat.gov.tw/pc/zh_TW/10/10002000300000000.html
http://referendum.2018.nat.gov.tw/pc/zh_TW/10/10002000400000000.html
http://referendum.2018.nat.gov.tw/pc/zh_TW/10/10002000500000000.html
http://referendum.2018.nat.gov.tw/pc/zh_TW/10/10002000600000000.html
http://referendum.2018.nat.gov.tw/pc/zh_TW/10/10002000700000000.html
http://referendum.2018.nat.gov.tw/

In [123]:
import pandas as pd

columns = ['公投案','縣市','鄉鎮市區','同意票數','不同意票數','有效票數','無效票數','投票數','投票權人數','投票率(%)','有效同意票數對投票權人數百分比(%)']
df = pd.DataFrame(data,columns=columns)

In [124]:
df

,公投案,縣市,鄉鎮市區,同意票數,不同意票數,有效票數,無效票數,投票數,投票權人數,投票率(%),有效同意票數對投票權人數百分比(%)
0,第7案,臺北市,松山區,"72,344","21,698","94,042","5,769","99,811","169,278",58.96%,42.74%
1,第7案,臺北市,信義區,"80,160","23,245","103,405","6,057","109,462","190,444",57.48%,42.09%
2,第7案,臺北市,大安區,"104,681","32,294","136,975","7,388","144,363","252,645",57.14%,41.43%
3,第7案,臺北市,中山區,"78,649","25,050","103,699","7,022","110,721","195,148",56.74%,40.3%
4,第7案,臺北市,中正區,"54,089","16,359","70,448","4,232","74,680","128,292",58.21%,42.16%
5,第7案,臺北市,大同區,"42,497","14,769","57,266","4,486","61,752","107,557",57.41%,39.51%
6,第7案,臺北市,萬華區,"64,385","19,585","83,970","6,622","90,592","164,020",55.23%,39.25%
7,第7案,臺北市,文山區,"101,174","26,288","127,462","7,798","135,260","227,211",59.53%,44.53%
8,第7案,臺北市,南港區,"43,217","12,009","55,226","3,158","58,384","101,907",57.29%,42.41%
9,第7案,臺北市,內湖區,"101,149","28,663","129,812","7,239","137,051","236,824",57.87%,42.71%


In [127]:
df.to_csv("referendum_107.csv", index=False)

In [128]:
df.to_pickle("referendum_107.pkl")